# Raw Data Processing

In [1]:
import os
import glob
import pathlib
import numpy as np
import pandas as pd
from numba import jit
import joblib
from joblib import Parallel, delayed
from rich.progress import track

from typing import Dict, List

import sys
sys.path.append("../src")

import config

from utils.common import timer
from utils.common import load_pickle, dump_pickle, save_cache
from utils.feature import FeatureStore

## Extract data each data_type from txt file

In [2]:
src_dir = pathlib.Path("../data/raw/train/")
filepaths = [
    path_filepath 
    for site_filepath in src_dir.glob("*") 
    for floor_filepath in site_filepath.glob("*") 
    for path_filepath in floor_filepath.glob("*")
]

In [58]:
filepath = filepaths[100]

site_id = filepath.parent.parent.name
floor = filepath.parent.name
path_id = filepath.name.split(".")[0]

feature = FeatureStore(
    site_id=site_id, 
    floor=floor, 
    path_id=path_id, 
    input_path="../data/raw/"
)
feature.load_all_data()

In [59]:
feature.save()

In [60]:
# attributes of site info
[c for c in dir(feature.site_info) if c[0]!="_"]

['draw_polygon',
 'features',
 'floor',
 'floor_coordinates',
 'get_site_info',
 'input_path',
 'map_type',
 'show_site_image',
 'site_height',
 'site_id',
 'site_width',
 'store_coordinates']

In [61]:
feature.site_info.site_height, feature.site_info.site_width

(212.0599746644009, 146.74902374734265)

In [62]:
feature.site_id, feature.floor, feature.path_id

('5da138764db8ce0c98bcaa46', 'F3', '5dabeed618410e00067e7027')

In [63]:
# attributes of feature store
for d_type in feature.df_types:
    print(d_type)
    display(feature[d_type].head())

accelerometer


,timestamp,x,y,z,accuracy
0,1571545625885,-0.950211,-0.447144,10.151520,2
1,1571545625905,-0.237320,-1.530518,10.049774,2
2,1571545625925,0.108643,-1.577805,10.280212,2
3,1571545625945,-0.507858,-0.368134,9.212997,2
4,1571545625965,-1.267441,-0.116745,9.645752,2


accelerometer_uncalibrated


,timestamp,x,y,z,x2,y2,z2,accuracy
0,1571545625885,-0.874176,-0.331619,10.253876,0.0,0.0,0.0,3
1,1571545625905,-0.803558,-0.672195,10.177856,0.0,0.0,0.0,3
2,1571545625925,0.302567,-2.148819,10.151520,0.0,0.0,0.0,3
3,1571545625945,-0.196625,-0.803879,9.709793,0.0,0.0,0.0,3
4,1571545625965,-1.028015,-0.207718,9.243530,0.0,0.0,0.0,3


beacon


...

,timestamp,x,y,z,accuracy
0,1571545625885,0.004937,0.035247,-0.714563,3
1,1571545625905,0.007161,0.037377,-0.713370,3
2,1571545625925,-0.003907,0.040647,-0.719029,3
3,1571545625945,-0.013164,0.039219,-0.726202,3
4,1571545625965,-0.010029,0.035698,-0.727309,3


waypoint


,timestamp,x,y
0,1571545625766,76.337490,94.725075
1,1571545630298,80.892600,94.235695
2,1571545639307,81.752380,86.743670
3,1571545651586,90.152565,86.986360
4,1571545660099,97.861170,87.440080


wifi


,timestamp,ssid,bssid,rssi,frequency,last_seen_timestamp
0,1571545627643,76f476c4a48560ff0ccd16e084028125183dd7d9,a1c69ca657ce727e64e782206b7b02a8b2d71dc8,-57,2412,1571545622849
1,1571545627643,21725587b1e7ba069dbe3ed05ffa2f3f2115b1e0,44e146eabacf580907da22dbf5ab239b5d65f057,-60,2437,1571545626146
2,1571545627643,06725136dd0bfa6439cda80db53eb483d9e3dda7,89dd2f80e478afc75ee414afd689f8b208a46206,-64,2437,1571545623130
3,1571545627643,7dc74a44a08ceffd931fcb77cd7e7e4c007d7072,899070d20cca7399c67bf9e45239a06899de9c17,-65,2437,1571545598998
4,1571545627643,06725136dd0bfa6439cda80db53eb483d9e3dda7,a9b8c0f03b34021867bd38bace4ada15e8002d14,-67,2462,1571545626460


---

## Create map for label encode

In [9]:
src_dir = pathlib.Path("../data/raw/train/")
filepaths = [
    path_filepath 
    for site_filepath in src_dir.glob("*") 
    for floor_filepath in site_filepath.glob("*") 
    for path_filepath in floor_filepath.glob("*")
]

### site for build

In [10]:
def create_site_map():
    def get_site_id_from_feature_store(filepath):
        path_id = filepath.name.split(".")[0]

        feature = load_pickle(f"../data/working/{path_id}.pkl", verbose=False)
        return feature.site_id
    
    site_ids = Parallel(n_jobs=-1)(delayed(get_site_id_from_feature_store)(filepath) for filepath in track(filepaths))
    unique_site_ids = np.unique(site_ids)
    siteId_map = {site_id: i + 1 for i, site_id in enumerate(unique_site_ids)}
    return siteId_map

In [11]:
%%time
siteId_map = create_site_map()

Output()

CPU times: user 20.1 s, sys: 4.42 s, total: 24.6 s
Wall time: 2min 20s


In [12]:
dump_pickle("./tmp/map_site_ids.pkl", siteId_map)

Dump pickle to ./tmp/map_site_ids.pkl


### bssid of wifi

In [13]:
def get_bssid_from_feature_store(filepath):
    path_id = filepath.name.split(".")[0]
    
    feature = load_pickle(f"../data/working/{path_id}.pkl", verbose=False)
    uniques = feature.wifi.bssid.unique()
    if len(uniques) > 0:
        return uniques
    else:
        return np.array([])

In [14]:
%%time
bssid = Parallel(n_jobs=-1)(delayed(get_bssid_from_feature_store)(filepath) for filepath in track(filepaths))

bssid = np.concatenate(bssid, axis=0)
unique_bsid = np.unique(bssid)

bssid_map = {_bssid: i + 1 for i, _bssid in enumerate(bssid)}
dump_pickle("./tmp/map_bssid.pkl", bssid_map)

Output()

Dump pickle to ./tmp/map_bssid.pkl
CPU times: user 29.3 s, sys: 3.21 s, total: 32.6 s
Wall time: 1min 48s


---

## Simple feature engineering

In [15]:
src_dir = pathlib.Path("../data/raw/train/")
filepaths = [
    path_filepath 
    for site_filepath in src_dir.glob("*") 
    for floor_filepath in site_filepath.glob("*") 
    for path_filepath in floor_filepath.glob("*")
]

### Create waypoint

In [16]:
%%time
def create_waypoint(filepaths: List):
    def get_waypoint_from_featureStore(filepath):
        path_id = filepath.name.split(".")[0]

        feature = load_pickle(f"../data/working/{path_id}.pkl", verbose=False)
        wp = feature['waypoint']
        wp['site'] = feature.site_id
        wp['floor'] = feature.n_floor
        wp['path'] = feature.path_id
        if len(wp) > 0:
            return wp
        else:
            return pd.DataFrame([])

    waypoint = Parallel(n_jobs=-1)(delayed(get_waypoint_from_featureStore)(filepath) for filepath in track(filepaths))
    waypoint = pd.concat(waypoint, axis=0).reset_index(drop=True)
    waypoint = waypoint.sort_values(by=['path', 'timestamp']).reset_index(drop=True)
    return waypoint

waypoint = create_waypoint(filepaths)

Output()

CPU times: user 1min 20s, sys: 4.01 s, total: 1min 24s
Wall time: 2min 2s


In [17]:
dump_pickle('./tmp/train_waypoint.pkl', waypoint)

Dump pickle to ./tmp/train_waypoint.pkl


In [18]:
waypoint.head()

,timestamp,x,y,site,floor,path
0,1558318437984,59.747032,244.61037,5cd969ba39e2fc0b4afe6fae,0,5ce215bc2d50640008bf22e2
1,1558318450619,55.360070,256.03390,5cd969ba39e2fc0b4afe6fae,0,5ce215bc2d50640008bf22e2
2,1558318459913,49.936070,254.65387,5cd969ba39e2fc0b4afe6fae,0,5ce215bc2d50640008bf22e2
3,1558318468827,44.060840,252.69990,5cd969ba39e2fc0b4afe6fae,0,5ce215bc2d50640008bf22e2
4,1558318550185,47.224000,245.42503,5cd969ba39e2fc0b4afe6fae,0,5ce215be915519000851776a


### Create build info

In [19]:
%%time
def create_build(filepaths: List):
    def get_waypoint_from_featureStore(filepath):
        path_id = filepath.name.split(".")[0]
        feature = load_pickle(f"../data/working/{path_id}.pkl", verbose=False)
        return site_map[feature.site_id], feature.site_info.site_height, feature.site_info.site_width
    
    site_map = load_pickle("./tmp/map_site_ids.pkl", verbose=False)
    resutls = Parallel(n_jobs=-1)(delayed(get_waypoint_from_featureStore)(filepath) for filepath in track(filepaths))
    return resutls

results = create_build(filepaths)

site_id, site_height, site_width = zip(*results)

Output()

CPU times: user 26.1 s, sys: 3.31 s, total: 29.4 s
Wall time: 1min 55s


In [20]:
site_id = np.array(site_id, dtype='int32')
site_height = np.array(site_height, dtype='float32')
site_width = np.array(site_width, dtype='float32')

In [21]:
site_id

array([199, 199, 199, ...,  96,  96,  96], dtype=int32)

### Create wifi 

In [22]:
%%time

def create_wifi():
    def get_wifi_feature(path_id, gdf):
        seq_len = 100
        bssid = []
        rssi = []
        freq = []

        feature = load_pickle(f"../data/working/{path_id}.pkl", verbose=False)
        wifi = feature.wifi.copy()
        wifi["bssid"] = wifi["bssid"].map(bssid_map)

        min_idx = gdf.index.min()
        max_idx = gdf.index.max()

        for i, row in gdf.iterrows():
            ts_pre_wp = gdf.loc[i - 1, "timestamp"] if i > min_idx else None
            ts_current_wp = gdf.loc[i, "timestamp"]
            ts_post_wp = gdf.loc[i + 1, "timestamp"] if (i + 1) < max_idx else None

            _wifi = wifi.copy()
            # NOTE: ターゲットとなるwaypointとその前後のwaypointの間にあるデータを取得する。
            ts_wifi = _wifi["timestamp"].values
            pre_flag = (
                np.ones(len(ts_wifi)).astype(bool)
                if ts_pre_wp == None
                else (ts_pre_wp < ts_wifi)
            )
            psot_flag = (
                np.ones(len(ts_wifi)).astype(bool)
                if ts_post_wp == None
                else (ts_wifi < ts_post_wp)
            )
            _wifi = _wifi[pre_flag & psot_flag]

            _wifi = _wifi.sort_values(by="rssi", ascending=False)
            _wifi = _wifi.head(seq_len)
            
            _bssid = np.zeros(seq_len)
            _rssi = np.tile(-999, seq_len)
            _freq = np.tile(-999, seq_len)
            
            _bssid[:len(_wifi)] = _wifi["bssid"].astype("int32").to_numpy()
            _rssi[:len(_wifi)] = _wifi["rssi"].astype("float32").to_numpy()
            _freq[:len(_wifi)] = _wifi["frequency"].astype("float32").to_numpy()
            
            bssid.append(_bssid)
            rssi.append(_rssi)
            freq.append(_freq)

        return bssid, rssi, freq

    waypoint = load_pickle("./tmp/train_waypoint.pkl", verbose=False)
    bssid_map = load_pickle("./tmp/map_bssid.pkl", verbose=False)
    results = Parallel(n_jobs=-1)(
        delayed(get_wifi_feature)(path_id, gdf)
        for path_id, gdf in track(waypoint.head(5000).groupby("path"))
    )
    return results

waypoint = load_pickle("./tmp/train_waypoint.pkl", verbose=False)
bssid_map = load_pickle("./tmp/map_bssid.pkl", verbose=False)

results = create_wifi()

Output()

CPU times: user 1min 1s, sys: 17.5 s, total: 1min 19s
Wall time: 2min 16s


In [23]:
bssid, rssi, freq = zip(*results)
bssid = np.concatenate(bssid, axis=0)
rssi = np.concatenate(rssi, axis=0)
freq = np.concatenate(freq, axis=0)

In [24]:
waypoint = load_pickle("./tmp/train_waypoint.pkl", verbose=False)

In [25]:
waypoint[0]

KeyError: 0

In [ ]:
feature = load_pickle(f"../data/working/{path_id}.pkl", verbose=False)